In [ ]:
pip install openpyxl

In [ ]:
%pip install webdriver-manager

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to c:\users\admin\appdata\local\temp\pip-req-build-azl67l0_
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit a058e90c9de41889c63bf2ee454bf1de064d70ff
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/haven-jeon/PyKoSpacing.git 'C:\Users\admin\AppData\Local\Temp\pip-req-build-azl67l0_'
ERROR: Could not find a version that satisfies the requirement tensorflow==2.9.3 (from pykospacing) (from versions: 2.12.0rc0)
ERROR: No matching distribution found for tensorflow==2.9.3


In [47]:
pip install konlpy

Note: you may need to restart the kernel to use updated packages.


In [62]:
import time
from openpyxl import Workbook
import pandas as pd
from bs4 import BeautifulSoup
#
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
#chrome driver 링크 연결 없이 바로 하는 법
#%pip install webdriver-manager (install받아야함)
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

#문자열에서 특수문자 제거
import re
#자동으로 엑셀 파일 열기위함(^^)
import openpyxl
#경고메세지 지워줌
import warnings 
warnings.filterwarnings('ignore')

#전처리
import konlpy
from konlpy.tag import Twitter

In [38]:
#엑셀 시트 생성
wb = Workbook(write_only=True)
ws = wb.create_sheet()


In [63]:
#크롬드라이버 링크 연결
# path='C:\chromedriver_win32\chromedriver.exe'
# driver= webdriver.Chrome(path) ->
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=Options())

#스크롤 시 풀 스크린으로 켜지게되면 우측에 '추천영상' 때문에 댓글보다 더 스크롤이 내려가게 됨
#하프 사이즈의 스크린으로 켜게 되면 추천영상이 일정이상 새로고침되지 않음 
driver.set_window_size(800, 1100)
#이 상태에서는 링크창에 data;,라고 적힌채로 대기상태에 들어감

In [64]:

#1. 검색어 입력받기
search=input()
driver.implicitly_wait(3)


#2. 유튜브 실행
driver.get("https://www.youtube.com")
#3초안에 웹페이지를 로딩하면 바로넘어가거나, 3초를 기다림
driver.implicitly_wait(3)

#get(링크)로 안하고 이것만 실행하면
#현재 페이지에 합쳐진다!!!

######검색#######


#3. 검색어 send
#ID,CLASS_NAME으로 하면 send가 안되거나 click조차 안될때가 있었음
#NAME으로 하면 둘 다 해결됨 이유는 모름?
search_text=driver.find_element(By.NAME,'search_query')

#로딩이 다 안됐기때문에 1초 기다려줌
driver.implicitly_wait(1)
# time.sleep(1)

#4. 검색어 전송
search_text.send_keys(search+" -Shorts")
driver.implicitly_wait(5)

#검색버튼 누르기
# search_btn=driver.find_element(By.CLASS_NAME,'style-scope.ytd-searchbox')
# search_btn.click()
# driver.implicitly_wait(3)

#5. 검색(엔터 누르기)
search_text.send_keys(Keys.ENTER)

In [ ]:
######영상 선정######
id='dismissible'



In [65]:
######스크롤하기######
#댓글 전체 스크랩하기 위해

#275개의 댓글 >> 23.5s , 23.2s , 24.0s 23~24s


#화면 최하단으로 스크롤 이동 : scrollTo(0, document.body.scrollHeight)
#자바 스크립트 문법 >execute_script(자바스크립트를 실행) > scrollTo

#window.scrollTo(x-좌표, y-좌표)
#시작위치지정(댓글창이 시작되는 위치)
#화면사이즈를 반으로 줄이고나니 스크롤을 끝까지 내린 후에 댓글이 생김
driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
driver.implicitly_wait(10)

#스크롤 이전 높이
#화면 바깥으로 삐져나간 부분까지 포함해서 전체 글의 길이를 scrollHeight
last_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    #스크롤의 y좌표를 가장아래(scrollHeight)까지 내림
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    driver.ex

    #스크롤 후 높이 구하기
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    #끝까지 스크롤 한 뒤 멈추기
    if new_height == last_height:
        break
    last_height = new_height

driver.implicitly_wait(1.5)


In [6]:
######유튜브 팝업 닫기######

#find_element_by_css_selector는 과거의 속성 find_element를 쓰는게 나음
try:
    driver.find_element(By.CSS_SELECTOR,"#dismiss-button > a").click()
except:
    pass

In [60]:
######대댓글 토글 열기######
# replies = driver.find_element(By.CSS_SELECTOR,"#more-replies > a")
# replies = driver.find_elements(By.CSS_SELECTOR,"#more-replies>a")

#find_elements < 무려 s를 안해서틀렸었음 하하 
replies = driver.find_elements(By.CLASS_NAME,"more-button")

time.sleep(1)

for reply in replies:
    #click도 안됨 send_keys도 안됨
    driver.execute_script("arguments[0].click();", reply)
    time.sleep(1.5) 

# reply.click()
# reply.send_keys(Keys.ENTER)
# 이거 안됨 스크립트로 해결함 근데 11분 18초 걸림!!!!! 댓글 275개에
# 댓글 106개 2m 50.2s


In [66]:
######댓글 가져오기#######
#page_source
#브라우저에 보이는 그대로의 HTML, 크롬 개발자 도구의 Element 탭 내용과 동일.
html_source = driver.page_source
#BeautifulSoup(html문서,'html.parser')
soup = BeautifulSoup(html_source, 'html.parser')

#soup.select 조건에 맞는 태그를 여러개 가져옴 
#soup.select_one() 조건에 맞는 태그를 한개(여러개가 있어도 한개만)
#태그이름  .클래스이름  #아이디이름  상위태그>자식태그>자식태그
#아이디>태그.클래스이름
id_list = soup.select("div#header-author > h3 > #author-text > span")
comment_list = soup.select("yt-formatted-string#content-text")

# id_final = []
comment_final=[]



twt=Twitter()

for i in range(len(comment_list)):
    # temp_id = id_list[i].text  #아이디 리스트를 가져와서
    # temp_id = temp_id.replace('\n', '') #필요한 부분만 남기면
    # temp_id = temp_id.replace('\t', '')
    # temp_id = temp_id.replace('    ', '')
    # temp_id = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]","",temp_id)
    # id_final.append(temp_id) # 댓글 작성자 id_final이 생김

    temp_comment = comment_list[i].text #마찬가지
    #형태소 기준으로 분리
    temp_comment = twt.pos(temp_comment) 
#    temp_comment = temp_comment.replace('\n', ' ') #필요한 부분만 남기면
#   temp_comment = temp_comment.replace('\t', ' ')
#    temp_comment = temp_comment.replace('\r', ' ').split(" ") #여기서 split하면 1번ㅇ째 @ 찾을 수 있음

    # 파이썬 특정 요소가 있는 리스트 제거  => 이거 안됨 @들어있는 모든 요소가 삭제됨
    # temp_comment에 @머시깽이가 있으면 그 특정요소 지움  
    # if temp_comment has @ > remove/delete element
    # for i in range(len(temp_comment)):
    #     if '@' in temp_comment[i]:
    #         #i번째 temp_comment를 삭제하라
    #         del temp_comment[i]

  

    #특수문자 빼기  contains(포함 유무)/find(포함되어있으면 어디에있는지 인덱스 값을 주고 없으면 -1을 줌)
#    temp_comment = list(filter(lambda n:n.find('@')!=0,temp_comment))
    # for넘길복잡 lambda로 함수부분에 한본에 넣어서 할라고?


#     # 위에서 split해서 re.sub에 안들어감
#     # temp_comment = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]"," ",temp_comment).replace(u'\xa0',u'').split(" ")


#     #map filter함수 사용법 @특정? 
#     # map(function, iterable) 
#     # function 적용시킬 함수
#     # iterable 적용시킬 객체
#     temp_comment=list(map(lambda n:re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z]","",n).replace(u'\xa0',u''),temp_comment))

#     temp_comment = list(filter(None,temp_comment))
    comment_final.append(temp_comment)



# # 2차원 배열을 1차원 배열로
# # for element in comment_pocket:
# #     comment_final += element

# #->
# comment_final= sum(comment_final,[])

# #결과확인

# print("댓글 출력: ")
print(comment_final)




[[('1월', 'Number'), ('20일', 'Number'), ('트위치', 'Noun'), ('방송', 'Noun'), ('복귀', 'Noun'), ('도', 'Josa'), ('기대', 'Noun'), ('해주세요', 'Verb'), ('!', 'Punctuation'), ('\r\n', 'Foreign'), ('↓↓↓', 'Foreign'), ('\r\n', 'Foreign'), ('https://www.twitch.tv/du_gan', 'URL')], [('우', 'Adverb'), ('와우', 'Noun'), ('저', 'Noun'), ('~', 'Punctuation'), ('믹스', 'Noun'), ('했어요', 'Verb'), ('~', 'Punctuation'), ('두', 'Determiner'), ('간', 'Noun'), ('님', 'Suffix'), ('여름', 'Noun'), ('푸름', 'Adjective'), ('대박', 'Noun'), ('이다', 'Josa'), ('그냥', 'Noun'), ('한겨울', 'Noun'), ('을', 'Josa'), ('여름', 'Noun'), ('으로', 'Josa'), ('만', 'Noun'), ('들어줬어', 'Verb'), ('덥다', 'Adjective'), ('더워', 'Adjective')], [('두', 'Determiner'), ('간', 'Noun'), ('님', 'Suffix'), ('의', 'Josa'), ('시원시원한', 'Adjective'), ('목소리', 'Noun'), ('는', 'Josa'), ('늘', 'Noun'), ('새', 'Noun'), ('롭고', 'Josa'), ('기쁜', 'Adjective'), ('마음', 'Noun'), ('밖에', 'Josa'), ('없어요', 'Adjective'), ('.', 'Punctuation'), ('항상', 'Noun'), ('두', 'Determiner'), ('간', 'Noun'), ('님', 'Suffix

In [40]:
######추출한 아이디 댓글 저장하기######
# pd_data = {"아이디" : id_final , "댓글 내용" : comment_final}
pd_data = {"댓글내용" : [comment_final]}

#pandas DataFrame만들기 =Table형식의 데이터를 다룰 때 사용한다.
youtube_pd = pd.DataFrame(pd_data)
print(pd_data)
youtube_pd.drop([1], axis=0, inplace=True)


youtube_pd.to_excel(search+'.xlsx')

{'댓글내용': [['1월', '20일', '트위치', '방송', '복귀도', '기대해주세요', 'httpswwwtwitchtvdugan', '우와우', '저', '믹스했어요', '두간님', '여름푸름', '대박이다', '그냥', '한겨울을', '여름으로', '만들어줬어', '덥다', '더워', '두간님의', '시원시원한', '목소리는', '늘', '새롭고', '기쁜', '마음밖에', '없어요', '항상', '두간님', '노래도', '들으면서', '힘을', '엄청', '받고는', '하는데', '이렇게', '청량한', '청춘', '여름', '곡을', '가지고', '오시다니', '겨울이어도', '너무', '더워지는', '기분이에요', '노래에서', '바다', '냄새', '나는건', '또', '새롭네요', '일러스트도', '너무', '좋고', '예뻐요', '그래도', '두간님이', '가장', '예쁘시고', '멋져요', '듣는동안', '계속', '집중했는데', '한동안', '계속', '듣겠네요', '또', '오랜만에', '발걸음', '해주셔서', '너무', '기뻐요', '트위치', '방송도', '시간이', '걸려서', '계속', '못', '갔는데', '이번에는', '꼭', '봐야겠어요', '정말', '너무', '수고하셨고', '좋은', '노래', '선사해주셔서', '감사드려요', '항상', '응원해요', '두간님', '한겨울을', '그냥', '여름으로', '바꿔버리시네요', '캬', '돌아오시는군요', '고생많으셨습니다', '안들었는데도', '벌써', '좋을거같네요', '여름이었다', '두간님', '노래', '진짜', '너무', '좋아요', '저', '사클도', '맨날', '들어요', '목소리사기', '노래실력사기', '최근에', '두간님', '생각하고', '있었는데', '이렇게', '나오는군요', '그동안', '일본노래', '많이', '들었는데', '전에', '몰랐던', '띵곡들을', '커버하셨더라고요', '이번에', '정주행', '갑니다', '무려', '3주전에도

KeyError: '[1] not found in axis'